# --- Day 15: Warehouse Woes ---
https://adventofcode.com/2024/day/15

In [72]:
def getRobotPlans():
    with open("robotPlans.txt") as file:
        return file.read()

In [73]:
def printMap(rmap):
    for i in range(len(rmap)):
        for j in range(len(rmap[i])):
            print(rmap[i][j], end="")
        print()
    print()

# Formatting
robotPlans = getRobotPlans()
# Get the robot map
rmap = robotPlans.split("\n\n")[0]
rmap = [[*x] for x in rmap.split("\n")]
rows, cols = len(rmap), len(rmap[0])
# Get the movements
movements = robotPlans.split("\n\n")[1]
movements = movements.replace("\n", "")

# Find robot
for i in range(rows):
    for j in range(cols):
        if rmap[i][j] == "@":
            robotCoords = [i, j]

# If obstacle in the way, call moveRobot on the object
# If object/robot is able to move, move it and return true
# If next object is a wall return false
def moveRobot(rmap, coords, direction) -> bool:
    nextMove = coords.copy()

    # Get the accurate next move
    if direction == "^":
        nextMove[0] -= 1
    elif direction == "v":
        nextMove[0] += 1
    elif direction == "<":
        nextMove[1] -= 1
    elif direction == ">":
        nextMove[1] += 1

    # Check to see if the robot/object cannot move
    if rmap[nextMove[0]][nextMove[1]] == "#":
        return False
    # If the robot/object can move, then move it and return true
    elif rmap[nextMove[0]][nextMove[1]] == ".":
        # Swap places
        rmap[nextMove[0]][nextMove[1]], rmap[coords[0]][coords[1]] = rmap[coords[0]][coords[1]], rmap[nextMove[0]][nextMove[1]]
        return True
    # If the next move is an object, call this function on the next move
    # If it returns True then that means it can and has moved, so move the current object too
    elif rmap[nextMove[0]][nextMove[1]] == "O":
        if moveRobot(rmap, nextMove, direction):
            # Swap places with the newly opened space
            rmap[nextMove[0]][nextMove[1]], rmap[coords[0]][coords[1]] = rmap[coords[0]][coords[1]], rmap[nextMove[0]][nextMove[1]]
            return True
        return False

# Loop through every movement in the instructions
for move in movements:
    # Move the robot if it can move
    canMove = moveRobot(rmap, robotCoords, move)
    if canMove:
        # If it was able to move then update the robot coordinates based on the direction
        # Get the accurate next move
        if move == "^":
            robotCoords[0] -= 1
        elif move == "v":
            robotCoords[0] += 1
        elif move == "<":
            robotCoords[1] -= 1
        elif move == ">":
            robotCoords[1] += 1

# Calculate the GPS coordinate sum
GPSCoordsSum = 0
for i in range(rows):
    for j in range(cols):
        if rmap[i][j] == "O":
            GPSCoordsSum += (i * 100) + j

print(f"Sum of all boxes' GPS coordinates: {GPSCoordsSum}")

Sum of all boxes' GPS coordinates: 1465523


# --- Part Two ---

In [103]:
# Print out map for testing purposes
def printMap(rmap):
    for i in range(len(rmap)):
        for j in range(len(rmap[i])):
            print(rmap[i][j], end="")
        print()
    print()

# Formatting
robotPlans = getRobotPlans()
robotPlans = """#######
#...#.#
#.....#
#..OO@#
#..O..#
#.....#
#######

<<<<<>"""#<vv<<^^<<^^
# Get the robot map
rmap = robotPlans.split("\n\n")[0]
rmap = rmap.split("\n")
# Make everything twice as wide
for row in range(len(rmap)):
    rmap[row] = rmap[row].replace(".", "..").replace("#", "##").replace("O", "[]").replace("@", "@.")
    rmap[row] = [*rmap[row]]
rows, cols = len(rmap), len(rmap[0])

# Get the movements
movements = robotPlans.split("\n\n")[1]
movements = movements.replace("\n", "")

# Find robot
for i in range(rows):
    for j in range(cols):
        if rmap[i][j] == "@":
            robotCoords = [i, j]
print(f"Robot location: {robotCoords}")

# Shift all coordinates on a map in a certain direction
def shiftCoords(rmap, affectedCoords, direction):
    
    # Loop until we've shifted all coordinates
    while affectedCoords:
        # Loop through each coordinate
        # If the space it's supposed to move into is empty, swap them and pop from the list
        for coord in affectedCoords:
            if direction == "^":
                if rmap[coord[0] - 1][coord[1]] == ".":
                    rmap[coord[0] - 1][coord[1]], rmap[coord[0]][coord[1]] = rmap[coord[0]][coord[1]], rmap[coord[0] - 1][coord[1]]
                    affectedCoords.pop(affectedCoords.index(coord))
            elif direction == "v":
                if rmap[coord[0] + 1][coord[1]] == ".":
                    rmap[coord[0] + 1][coord[1]], rmap[coord[0]][coord[1]] = rmap[coord[0]][coord[1]], rmap[coord[0] + 1][coord[1]]
                    affectedCoords.pop(affectedCoords.index(coord))
            elif direction == "<":
                if rmap[coord[0]][coord[1] - 1] == ".":
                    rmap[coord[0]][coord[1] - 1], rmap[coord[0]][coord[1]] = rmap[coord[0]][coord[1]], rmap[coord[0]][coord[1] - 1]
                    affectedCoords.pop(affectedCoords.index(coord))
            elif direction == ">":
                if rmap[coord[0]][coord[1] + 1] == ".":
                    rmap[coord[0]][coord[1] + 1], rmap[coord[0]][coord[1]] = rmap[coord[0]][coord[1]], rmap[coord[0]][coord[1] + 1]
                    affectedCoords.pop(affectedCoords.index(coord))

# Find all coordinates that would be affected by the shift using BFS
# If we encounter a wall, we cannot shift, so return False
# If we finish the BFS then shift all explored nodes in the right direction
def moveRobot(rmap, coords, direction) -> bool:

    # Create a frontier and explored nodes list
    frontier = [coords]
    explored = [coords]

    # Loop through all points until frontier is empty
    while frontier:
        # Get the current coordinates
        currentCoords = frontier.pop(0)

        # Get the next coordinate to be searched
        if direction == "^":
            nextCoord = [currentCoords[0] - 1, currentCoords[1]]
        elif direction == "v":
            nextCoord = [currentCoords[0] + 1, currentCoords[1]]
        elif direction == "<":
            nextCoord = [currentCoords[0], currentCoords[1] - 1]
        elif direction == ">":
            nextCoord = [currentCoords[0], currentCoords[1] + 1]

        # If next coordinate is a wall, they cannot move so return False
        if rmap[nextCoord[0]][nextCoord[1]] == "#":
            return False
        # I think I'm just going to do nothing if we hit an empty space, but I'll keep this here for now
        elif rmap[nextCoord[0]][nextCoord[1]] == ".":
            pass
        elif rmap[nextCoord[0]][nextCoord[1]] == "[":
            # Check this node and the node to the right of it
            if nextCoord not in explored:
                frontier.append(nextCoord)
                explored.append(nextCoord)
            if [nextCoord[0], nextCoord[1] + 1] not in explored:
                frontier.append([nextCoord[0], nextCoord[1] + 1])
                explored.append([nextCoord[0], nextCoord[1] + 1])
        elif rmap[nextCoord[0]][nextCoord[1]] == "]":
            # Check this node and the node to the left of it
            if nextCoord not in explored:
                frontier.append(nextCoord)
                explored.append(nextCoord)
            if [nextCoord[0], nextCoord[1] - 1] not in explored:
                frontier.append([nextCoord[0], nextCoord[1] - 1])
                explored.append([nextCoord[0], nextCoord[1] - 1])

    # If we make it to the end, shift all affected cells (explored) in the correct direction
    # Return True to indicate that we can shift the robot too
    print(explored)
    shiftCoords(rmap, explored, direction)
    return True

printMap(rmap)
# Loop through every movement in the instructions
for move in movements:
    # Check the direction and move if it can be moved
    canMove = moveRobot(rmap, robotCoords, move)
    if canMove:
        # If it was able to move then update the robot coordinates based on the direction
        # Get the accurate next move
        if move == "^":
            robotCoords[0] -= 1
        elif move == "v":
            robotCoords[0] += 1
        elif move == "<":
            robotCoords[1] -= 1
        elif move == ">":
            robotCoords[1] += 1
    print(move)
    printMap(rmap)

Robot location: [3, 10]
##############
##......##..##
##..........##
##....[][]@.##
##....[]....##
##..........##
##############

[[3, 10], [3, 9], [3, 8], [3, 7], [3, 6]]
<
##############
##......##..##
##..........##
##...[][]@..##
##....[]....##
##..........##
##############

[[3, 9], [3, 8], [3, 7], [3, 6], [3, 5]]
<
##############
##......##..##
##..........##
##..[][]@...##
##....[]....##
##..........##
##############

[[3, 8], [3, 7], [3, 6], [3, 5], [3, 4]]
<
##############
##......##..##
##..........##
##.[][]@....##
##....[]....##
##..........##
##############

[[3, 7], [3, 6], [3, 5], [3, 4], [3, 3]]
<
##############
##......##..##
##..........##
##[][]@.....##
##....[]....##
##..........##
##############

<
##############
##......##..##
##..........##
##[][]@.....##
##....[]....##
##..........##
##############

[[3, 6]]
>
##############
##......##..##
##..........##
##[][].@....##
##....[]....##
##..........##
##############



In [ ]:
# Ideas - 
# Horizontal should work the same as part 1
# Vertical works very different
# Ideas for verticals - 
# Just move everything. If the border is replaced with something other than a "." then undo that change
# (BFS to find all points connected above?)

# OR!!!
# BFS all boxes above/below (depending on direction).
# If BFS touches "#" then don't do it. Otherwise push everything being touched up/down